In [3]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer
import re
import argparse

# parser = argparse.ArgumentParser(description='env term small or big')
# parser.add_argument("--small", default=False, type=bool, help="When true, run on smaller data")
# args = parser.parse_args()
small = True

environmentalTerms = [line.rstrip('\n') for line in open("../data/helper_files/environmentalTerms.txt")]
environmentalTerms = list(dict.fromkeys(environmentalTerms))
if small:
    restaurants_and_reviews_file = "../data/small_restaurants_and_reviews.csv"
    term_based_file = "../data/small_term_based_green_rating_results.csv"
    print("smallness achieved")
else:
    restaurants_and_reviews_file = "../data/big_restaurants_and_reviews.csv"
    term_based_file = "../data/big_term_based_green_rating_results.csv"
    print("we livin large")
restaurants_and_reviews = pd.read_csv(restaurants_and_reviews_file)

smallness achieved


In [4]:

def pre_process(text):
    # lowercase
    text = text.lower()

    # remove tags
    text = re.sub("&lt;/?.*?&gt;", " &lt;&gt; ", text)

    # remove special characters and digits
    text = re.sub("(\\d|\\W)+", " ", text)

    # splittext = text.split(' ')
    # lemmatizer = WordNetLemmatizer()

    # lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in splittext])

    return text  # lemmatized_output


restaurants_and_reviews['review_text'] = restaurants_and_reviews['review_text'].apply(lambda x: pre_process(x))


# def get_env_terms(text):
#     terms = environmentalTerms
#     #     text = text.split(' ')
#     env_terms_in_text = [term for term in terms if term in text]
#     return ', '.join(env_terms_in_text)


def get_env_term_counts(restaurant):
    terms = environmentalTerms
    env_terms_in_text = [term for term in terms if term in restaurant['review_text']]
    # sum = 0
    # for term in restaurant['env_terms'].split(", "):
    #     #         print(term , " count " , rest['text'].count(term))
    #     if term != '':
    #         sum += restaurant['text'].count(term)
    return len(env_terms_in_text)

In [5]:
restaurants_and_reviews['env_term_counts'] = restaurants_and_reviews.apply(
    lambda x: get_env_term_counts(x), axis=1)
restaurants_and_reviews.sort_values(['env_term_counts'], ascending=False, inplace=True)

restaurants_and_reviews['len'] = restaurants_and_reviews['review_text'].str.split().apply(len)

restaurants_and_reviews['env_terms_percent_of_overall_words'] = (
        restaurants_and_reviews['env_term_counts'] / restaurants_and_reviews['len'])
restaurants_and_reviews['env_terms_percent_of_overall_words'] *= 100

bins = [-1, 0.01, 0.3, 1, 100]
labels = [0, 1, 2, 3]
restaurants_and_reviews['term_based_green_rating'] = pd.cut(
    restaurants_and_reviews['env_terms_percent_of_overall_words'],
    bins=bins, labels=labels)

print(restaurants_and_reviews.head())
restaurants_and_reviews[['name', 'term_based_green_rating']].to_csv(
    term_based_file, index=False)


      Unnamed: 0             business_id  \
1057        2640  Wxxvi3LZbHNIDwJ-ZimtnA   
781         1975  ujHiaprwCQ5ewziu0Vi9rw   
1167        2944  R1jJQi2yR44D_2ileqr8kA   
2403        6051  XXW_OFaYQkkGOGniujZFHg   
2785        7051  LYWbyu1EQM2o5hwHG5_Ldg   

                                             categories  \
1057  SHOPPING CENTERS  RESORTS  ARTS & ENTERTAINMEN...   
781                AMERICAN (NEW)  BUFFETS  RESTAURANTS   
1167      RESTAURANTS  VEGAN  LIVE/RAW FOOD  VEGETARIAN   
2403  NIGHTLIFE  LOUNGES  RESTAURANTS  BARS  AMERICA...   
2785                   RESTAURANTS  SUSHI BARS  SEAFOOD   

                                             name  review_count  stars  \
1057                       THE VENETIAN LAS VEGAS          3499    4.0   
781                        THE BUFFET AT BELLAGIO          4227    3.5   
1167                               LIVING KITCHEN           417    4.5   
2403  THE PEPPERMILL RESTAURANT & FIRESIDE LOUNGE          3260    4.0   
2785      

In [6]:
restaurants_and_reviews['term_based_green_rating'].describe()

count     5000
unique       4
top          1
freq      3436
Name: term_based_green_rating, dtype: int64